In [1]:
import pandas as pd
from pandas import read_csv
import requests
import json
import math
import time
from datetime import datetime
import copy


In [16]:
## Remove dictionary keys for empty lists and null values
def cleanNullTerms(d):
    clean = {}   
    for k, v in d.items():
        if v is not None and len(v)>0:
            clean[k] = v   
    return clean

## Fetch the data from biotools
def get_pages(biotoolsapiurl):
    biotoolsapiurl
    payloads = {'format': 'json', 'domain': 'covid-19','page':'i'}
    r = requests.get(biotoolsapiurl, params=payloads, timeout=20).json()
    count = r['count']
    list_num = len(r['list'])
    total_pages = math.ceil(count/list_num)
    return total_pages

## Pull the key for dictionaries with only one key-value pair
def get_dict_key(single_dict):
    keylist = list(single_dict.keys())
    only_key = keylist[0]
    return only_key

## Parse defined terms and format them as a Defined Term object
def parse_defined_terms(definedtermlist):
    try:
        termdf = pd.DataFrame(definedtermlist)
        termdf.rename(columns={'uri':'url','term':'name'},inplace=True)
        termdf['inDefinedTermSet']= 'http://edamontology.org/'
        termdf['@id'] = termdf['url'].str.replace('http://edamontology.org/','')
        termdf['@type'] = 'schema:DefinedTerm'
        termdf['termCode'] = termdf['@id']
        cleandf = termdf[['@type','@id','inDefinedTermSet','termCode','url','name']]
        termjson = cleandf.to_dict(orient="records")
    except:
        termjson = -1
    return termjson

## Parse input and outputs and format as a list of Formal Parameter objects
def parse_parameters(parameterdict):
    datalist = []
    formatlist = []
    for eachparameter in parameterdict:
        content_types = list(eachparameter.keys())
        if 'data' in content_types:
            if isinstance(eachparameter['data'], dict) == True:
                datadictlist = []
                datadictlist.append(eachparameter['data'])
            elif isinstance(eachparameter['data'], list) == True:
                datadictlist = eachparameter['data']
            datavalue = parse_defined_terms(datadictlist)
            if datavalue != -1:
                datalist.extend(datavalue)               
        if 'format' in content_types:
            if isinstance(eachparameter['format'], dict) == True:
                formatdictlist = []
                formatdictlist.append(eachparameter['format'])
            elif isinstance(eachparameter['format'], list) == True:
                formatdictlist = eachparameter['format']
            formatvalue = parse_defined_terms(formatdictlist)
            if formatvalue != -1:
                formatlist.extend(formatvalue)
    if len(datalist) > 0 or len(formatlist) > 0:
        parameterjson = {'@type':'bioschemastypes:FormalParameter',
                         'encodingFormat': datalist,
                         'defaultValue': formatlist}
    else:
        parameterjson = -1
    return parameterjson

## Create an outbreak-compatible json
class cleandoc:
    now = datetime.now()
    basejson = {
      "@type": "outbreak:ComputationalTool",
      "@context": {
        "schema": "http://schema.org/",
        "outbreak": "https://discovery.biothings.io/view/outbreak/",
        "dct": "http://purl.org/dc/terms/"
      },
      "curatedBy": {
        "@type": "outbreak:Organization",
        "name": "ELIXIR bio.tools",
        "url": "https://bio.tools/t?domain=covid-19",
        "identifier": "biotools",
        "affiliation": [{"@type":"outbreak:Organization","name": "ELIXIR"}],
        "curationDate": now.strftime("%Y-%m-%d")
      }
    }
    
    ## Directly add any key/value pair that has minimal transformations
    def add_basic_info(biotooljsonhit): 
        cleanjson = cleandoc.basejson
        cleanjson['name'] = biotooljsonhit['name']
        cleanjson['description'] = biotooljsonhit['description']
        cleanjson['identifier'] = biotooljsonhit['biotoolsID']
        cleanjson['url'] = biotooljsonhit['homepage']
        cleanjson['softwareVersion'] = biotooljsonhit['version']
        cleanjson['applicationCategory'] = biotooljsonhit['toolType']
        cleanjson['license'] = biotooljsonhit['license']
        cleanjson['programmingLanguage'] = biotooljsonhit['language']
        if biotooljsonhit['accessibility'] != None:
            cleanjson['conditionsOfAccess'] = biotooljsonhit['accessibility']
        elif biotooljsonhit['cost'] != None:
            cleanjson['conditionsOfAccess'] = biotooljsonhit['cost']
        try:
            cleanjson['dateModified'] = biotooljsonhit['lastUpdate'].split('T')[0]
        except:
            cleanjson['dateModified'] = biotooljsonhit['lastUpdate']
        try:
            cleanjson['dateCreated'] = biotooljsonhit['additionDate'].split('T')[0]
        except:
            cleanjson['dateCreated'] = biotooljsonhit['additionDate']          
        return cleanjson

    ## Add application subcategory after transforming it
    def add_app_sub_cat(cleanjson,biotooljsonhit): 
        try:
            alltopics = biotooljsonhit['topic']
            topicjson = parse_defined_terms(alltopics)
            if topicjson != -1:
                cleanjson['applicationSubCategory'] = topicjson
        except:
            pass
        return cleanjson
    
    ## Add featuress after transforming them        
    def add_features(cleanjson,biotooljsonhit):
        available_functions = biotooljsonhit['function']
        if len(available_functions) > 0:
            operationlist = []
            inputlist = []
            outputlist = []
        for eachfunction in available_functions:
            ## parse operations
            operations = eachfunction['operation']
            features = parse_defined_terms(operations)
            if features != -1:
                operationlist.extend(features)
            ## parse inputs
            inparameterdict = eachfunction['input']
            inparameters = parse_parameters(inparameterdict)
            if inparameters != -1:
                inputlist.append(inparameters)
            ## parse outputs
            outparameterdict = eachfunction['output']
            outparameters = parse_parameters(outparameterdict)
            if outparameters != -1:
                outputlist.append(outparameters)
                ## Note, additional content available from bio.tools in this section includes 'cmd', and 'notes'
            if len(operationlist)>0:
                cleanjson['featureList'] = operationlist
            if len(inputlist)>0:
                cleanjson['input'] = inputlist
            if len(outputlist)>0:
                cleanjson['output'] = outputlist
        return cleanjson

    ## get the downloadUrl and codeRepository from either link or download
    def add_links(cleanjson,biotooljsonhit): 
        ## pool links
        biotoollinks = []
        if isinstance(biotooljsonhit['link'],list)==True:
            for eachdict in biotooljsonhit['link']:
                biotoollinks.append(eachdict)
        if isinstance(biotooljsonhit['download'],list)==True:
            for eachdict in biotooljsonhit['link']:
                biotoollinks.append(eachdict)
        if isinstance(biotooljsonhit['link'],dict)==True:
            biotoollinks.append(biotooljsonhit['link'])
        if isinstance(biotooljsonhit['download'],dict)==True:
            biotoollinks.append(biotooljsonhit['download'])

        ## Parse links
        if len(biotoollinks) > 0:
            codeRepository = []
            discussionUrl = []
            downloadUrl = []
            for eachitem in biotoollinks:
                if isinstance(eachitem,dict)==True and eachitem['type']!=None:
                    if ('Repository' or 'repository') in eachitem['type']:
                        codeRepository.append(eachitem['url'])
                    if ('Issue' or 'issue') in eachitem['type']:
                        discussionUrl.append(eachitem['url'])
                    if ('file' in eachitem['type']):
                        downloadUrl.append(eachitem['url'])
                    if ('note' in eachitem.keys()) and (eachitem['note']!=None) and ('image' in eachitem['note']):
                        downloadUrl.append(eachitem['url'])
            if len(codeRepository)>0:
                cleanjson['codeRepository'] = list(set(codeRepository))
            if len(discussionUrl)>0:
                cleanjson['discussionUrl'] = list(set(discussionUrl))
            if len(downloadUrl)>0:
                cleanjson['downloadUrl'] = list(set(downloadUrl))
        return cleanjson
    
    ## parse softwarehelp from documentation
    def add_softwarehelp(cleanjson,biotooljsonhit):
        if len(biotooljsonhit['documentation'])>0:
            if isinstance(biotooljsonhit['documentation'],list)==True:
                cleanjson['softwareHelp'] = [x['url'] for x in biotooljsonhit['documentation']]
            if isinstance(biotooljsonhit['documentation'],dict)==True:
                cleanjson['softwareHelp'] = [biotooljsonhit['documentation']]
        return cleanjson

    ## parse authors from credit
    def add_author(cleanjson,biotooljsonhit):
        authorlist = []
        for eachhit in biotooljsonhit['credit']:
            newdict = copy.deepcopy(eachhit)
            if newdict['typeEntity'] == 'outbreak:Person':
                newdict['@type'] = 'outbreak:Person'
            elif newdict['typeEntity'] == None: 
                if newdict['orcidid']!= None:
                    newdict['@type'] = 'outbreak:Person'
                else:
                    newdict['@type'] = 'dct:Agent'
            else:
                newdict['@type'] = 'outbreak:Organization'
            newdict['identifier'] = newdict.pop('orcidid',None)
            newdict['role'] = newdict.pop('typeRole',None)
            newdict.pop('gridid',None)
            newdict.pop('rorid',None)
            newdict.pop('fundrefid',None)
            newdict.pop('note',None)
            newdict.pop('typeEntity',None)
            for key, value in dict(newdict).items():
                if (value is None) or (len(value)==0):
                    del newdict[key]
            authorlist.append(newdict)
        cleanjson['author'] = authorlist
        return cleanjson
    
    ## parse citedBy from 'publication'
    def add_citations(cleanjson,biotooljsonhit):
        citedby = []
        for eachpub in biotooljsonhit['publication']:
            tmppub = copy.deepcopy(eachpub)
            tmppub['@type'] = 'outbreak:Publication'
            try:
                tmppub['name'] = tmppub['metadata']['title']
            except:
                tmppub['name'] = None
            tmppub.pop('type')
            tmppub.pop('version')
            tmppub.pop('note')
            tmppub.pop('metadata')
            for key, value in dict(tmppub).items():
                if (value is None) or (len(value)==0):
                    del tmppub[key]
            citedby.append(tmppub)
        cleanjson['citedBy'] = citedby
        
        return cleanjson
    


In [19]:
## Fetch the json docs from the API
def download_jsondocs():
    jsondoclist = []
    biotoolsapiurl = 'https://bio.tools/api/t'
    payloads = {'format': 'json', 'domain': 'covid-19','page':'1'}
    r = requests.get(biotoolsapiurl, params=payloads, timeout=5).json()
    count = r['count']
    list_num = len(r['list'])
    total_pages = math.ceil(count/list_num) 
    i=1
    while i < total_pages+1:
        payloads = {'format': 'json', 'domain': 'covid-19','page':i}
        r = requests.get(biotoolsapiurl, params=payloads, timeout=20).json()
        time.sleep(1)
        jsondoclist.extend(r['list'])
        i=i+1
    return jsondoclist

    
## Transform the json docs
def transform_json(jsondoclist):
    for i in range(len(jsondoclist)):
        biotooljsonhit = jsondoclist[i]
        cleanjson = cleandoc.add_basic_info(biotooljsonhit)
        cleanjson = cleandoc.add_app_sub_cat(cleanjson,biotooljsonhit)
        cleanjson = cleandoc.add_features(cleanjson,biotooljsonhit)
        cleanjson = cleandoc.add_links(cleanjson,biotooljsonhit)
        cleanjson = cleandoc.add_softwarehelp(cleanjson,biotooljsonhit)
        cleanjson = cleandoc.add_author(cleanjson,biotooljsonhit)
        cleanjson = cleandoc.add_citations(cleanjson,biotooljsonhit)    
        yield cleanNullTerms(cleanjson)
        

def load_annotations():
    jsondoclist = download_jsondocs()
    doclist = transform_json(jsondoclist)
    yield from doclist

### Check Functions

In [4]:
biotoolsapiurl = 'https://bio.tools/api/t'
payloads = {'format': 'json', 'domain': 'covid-19','page':'1'}
#payloads = {'format': 'json', 'q': 'COVID-19','page':'1'}
r = requests.get(biotoolsapiurl, params=payloads, timeout=5).json()
count = r['count']
list_num = len(r['list'])
total_pages = math.ceil(count/list_num)
print(list_num)
print(count)
print(total_pages)

10
206
21


In [ ]:
i=1
while i < total_pages+1:
    payloads = {'format': 'json', 'domain': 'covid-19','page':i}
    r = requests.get(biotoolsapiurl, params=payloads, timeout=20).json()
    time.sleep(1)
    i=i+1

In [14]:
print(r['list'][4]['credit'])
biotooljsonhit = r['list'][1]

[{'name': 'Bridget P Bannerman', 'email': 'bpc28@cam.ac.uk', 'url': None, 'orcidid': 'https://orcid.org/0000-0002-5746-8283', 'gridid': None, 'rorid': None, 'fundrefid': None, 'typeEntity': 'Person', 'typeRole': [], 'note': None}, {'name': 'R Andres Floto', 'email': 'arf27@cam.ac.uk', 'url': None, 'orcidid': 'https://orcid.org/0000-0002-2188-5659', 'gridid': None, 'rorid': None, 'fundrefid': None, 'typeEntity': 'Person', 'typeRole': [], 'note': None}, {'name': 'Jorge Júlvez', 'email': None, 'url': None, 'orcidid': 'https://orcid.org/0000-0002-7093-228X', 'gridid': None, 'rorid': None, 'fundrefid': None, 'typeEntity': None, 'typeRole': [], 'note': None}, {'name': 'Alexandru Oarga', 'email': None, 'url': None, 'orcidid': 'https://orcid.org/0000-0002-7271-733X', 'gridid': None, 'rorid': None, 'fundrefid': None, 'typeEntity': None, 'typeRole': [], 'note': None}, {'name': 'Tom L Blundell', 'email': None, 'url': None, 'orcidid': None, 'gridid': None, 'rorid': None, 'fundrefid': None, 'typeEn

In [ ]:
print(biotooljsonhit.keys())

In [ ]:
## parse doi from 'otherID' (if available)
biotooljsonhit['otherID']

In [17]:
cleanjson = transform_json(r['list'])
for eachjson in cleanjson:
    print(eachjson)

{'@type': 'outbreak:ComputationalTool', '@context': {'schema': 'http://schema.org/', 'outbreak': 'https://discovery.biothings.io/view/outbreak/'}, 'curatedBy': {'@type': 'outbreak:Organization', 'name': 'ELIXIR bio.tools', 'url': 'https://bio.tools/t?domain=covid-19', 'identifier': 'biotools', 'affiliation': [{'@type': 'outbreak:Organization', 'name': 'ELIXIR'}], 'curationDate': '2022-08-04'}, 'name': 'DisGeNET', 'description': 'DisGeNET is a discovery platform containing one of the largest publicly available collections of genes and variants associated to human diseases. DisGeNET integrates data from expert curated repositories, GWAS catalogues, animal models and the scientific literature. DisGeNET data are homogeneously annotated with controlled vocabularies and community-driven ontologies. Additionally, several original metrics are provided to assist the prioritization of genotype–phenotype relationships.\n\nThe current version of DisGeNET (v7.0) contains 1,134,942 gene-disease asso

### Run time check

In [25]:
%%time

jsondoclist = download_jsondocs()
print(len(jsondoclist))

206
Wall time: 38.8 s


In [24]:
%%time

doclist = transform_json(jsondoclist)

i=0
for doc in doclist:
    i=i+1
print(i)

206
Wall time: 1.77 s


### Integrated, and no longer needed

In [ ]:

def add_version(cleanjson, biotooljsonhit):
    try:
        cleanjson['softwareVersion'] = biotooljsonhit['version']
    except:
        pass
    return(cleanjson) 
   
    
def add_application_cat(cleanjson, biotooljsonhit):
    try:
        cleanjson['applicationCategory'] = biotooljsonhit['toolType']
    except:
        pass
    return(cleanjson)

In [ ]:
cleanjson = cleandoc.add_basic_info(biotooljsonhit)
#print(cleanjson)
cleanjson = cleandoc.add_app_sub_cat(cleanjson,biotooljsonhit)
#print(cleanjson)
cleanjson = cleandoc.add_features(cleanjson,biotooljsonhit)
#print(cleanjson)
cleanjson = cleandoc.add_links(cleanjson,biotooljsonhit)
#print(cleanjson)
cleanjson = cleandoc.add_softwarehelp(cleanjson,biotooljsonhit)
#print(cleanjson)
cleanjson = cleandoc.add_author(cleanjson,biotooljsonhit)
#print(cleanjson)
cleanjson = cleandoc.add_citations(cleanjson,biotooljsonhit)
print(cleanjson)
